# Possibly useful tools

**how many times something occurs in a column**

In [ ]:
for column in climate_data.columns:
  if column.startswith('STATION'):
    results = climate_data[column].value_counts(dropna=False).to_dict() #special way to count 'NaN' values
    print(f"Column '{column}' counts: {results}")
    print()


**combining CSV files**

In [ ]:
import glob

# Step 1: Set the path to your CSV files
csv_files_path = '/content/combine/*.csv'

# Step 2: Use glob to list all CSV files in the directory
csv_files = glob.glob(csv_files_path)

# Step 3: Initialize an empty list to hold DataFrames and a flag for consistency check
data_frames = []
consistent = True

# Step 4: Loop through the list of CSV files and read each one into a DataFrame
for file in csv_files:
    df = pd.read_csv(file)
    data_frames.append(df)

# Step 5: Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(data_frames, ignore_index=True)

# Step 6: (Optional) Save the combined DataFrame to a new CSV file
combined_df.to_csv('combined_weather_dataNEW.csv', index=False)

# Step 7: Print the shape of the combined DataFrame to verify
print('Combined DataFrame shape:', combined_df.shape)



# Step 4: Loop through the list of CSV files and read each one into a DataFrame
for file in csv_files:
    try:
        df = pd.read_csv(file)

        # Check if all rows in the current CSV file are the same
        inconsistent_columns = df.columns[~df.eq(df.iloc[0]).all()]
        if not inconsistent_columns.empty:
            consistent = False
            print(f"Inconsistency found in file: {file}")
            print(f"Inconsistent columns: {list(inconsistent_columns)}")

        data_frames.append(df)

    except Exception as e:
        print(f"Error reading file {file}: {e}")
        consistent = False

if consistent:
    # Step 5: Concatenate all DataFrames into a single DataFrame
    combined_df = pd.concat(data_frames, ignore_index=True)

    # Step 6: (Optional) Save the combined DataFrame to a new CSV file
    combined_df.to_csv('combined_weather_data.csv', index=False)

    # Step 7: Print the shape of the combined DataFrame to verify
    print('Combined DataFrame shape:', combined_df.shape)
else:
    print("Not all CSV files are consistent. Please check the reported files.")

**counting number of non-null values in every column (sorted)**

In [ ]:
# Get non-null counts for each column
non_null_counts = climate_data.notnull().sum()

# Create a summary DataFrame
summary_df = pd.DataFrame({
    'Column': non_null_counts.index,
    'Non-Null Count': non_null_counts.values
})

# Sort the summary DataFrame by 'Non-Null Count' in descending order
sorted_summary_df = summary_df.sort_values(by='Non-Null Count', ascending=False)

# Set pandas display options to prevent truncation
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Display the sorted summary
print(sorted_summary_df)

# delete later?

In [ ]:
# Filling NaN values with the mean of each numeric column
for col in climate_data.columns:
    if col not in exclude_columns and pd.api.types.is_numeric_dtype(climate_data[col]):
        climate_data[col] = climate_data[col].fillna(climate_data[col].mean())

### /other ways to handle NaN values ###
'''
# (NOT RECOMMENDED) Dropping rows with NaN values:
climate_data = climate_data.dropna()

# Replacing NaN values with 0:
climate_data = climate_data.fillna(0)
'''

In [ ]:
# Loading the data
pd.set_option('display.max_columns', None)
#climate_data = pd.read_csv("/content/2024_test_3738819.csv") #test data
climate_data_original = pd.read_csv("/content/combined_weather_data.csv")
climate_data = climate_data_original.copy()

climate_data['DATE'] = pd.to_datetime(climate_data['DATE'], format='%Y-%m-%dT%H:%M:%S') # Changing the time format
climate_data.set_index('DATE', inplace=True) # Setting the dates as the indices (directly changing the file)
#climate_data.tail()